## Cel·la de prova

ha de sortir una ruta que inclogui envs/myenv

In [1]:
import sys
print(sys.executable)

c:\Users\UX433\miniconda3\envs\myenv\python.exe


# Crear blueprint

### Comprobació preliminar del directori i kernell

Per verificar que datasets.ods i el notebook estan a la llista

In [2]:
import os, sys
print("Python executable:", sys.executable)     # mostra el Python que corre (ha de ser el env activat)
print("Directori de treball:", os.getcwd())    # carpeta del repo
print("Llistat d'arxius a la carpeta:")
for f in sorted(os.listdir('.')):
    print(" -", f)

Python executable: c:\Users\UX433\miniconda3\envs\myenv\python.exe
Directori de treball: c:\Users\UX433\OneDrive\Escritorio\networkalignment
Llistat d'arxius a la carpeta:
 - .git
 - A2_blueprint.npy
 - A_test_equiv.npy
 - C-el_k4_Synthetic- No groups - Simoultenously.ipynb
 - codienbrut.ipynb
 - crear_blueprint.ipynb
 - datasets.ods
 - environment.yml
 - synthetic_paper.npy


In [3]:
import itertools
import pickle
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import os
import time
from numba import jit, njit
from numba.types import bool_, int_, float32
from math import comb
from copy import deepcopy
from tqdm import tqdm
import networkx as nx
import pandas as pd
from collections import defaultdict
from itertools import permutations

### Llegir fulls de càlcul 

In [4]:
### Open the files
fulls = 8
#n_grupo = 4 - ara no estic fent servir grups

d = {} #crea un diccionari per emmagatzemar DataFrames(taules de memòria)
D = "Dataset" #prefix del nom de les fulles — al excel es diuen Dataset1, Dataset2, ...
for i in range(0,fulls): #recorro 0...7
    d["group" + str(i)] = pd.read_excel("datasets.ods", sheet_name=D+str(i+1)) #el que al diccionari serà gorup 0, sera el meu Dataset1
    # hem assignat al diccionari: d["group0"] = <taula Dataset1> 
    print('Read:',i, 'size', d["group" + str(i)].shape) #volem imprimir el nre de files i columnes

Read: 0 size (227, 183)
Read: 1 size (227, 183)
Read: 2 size (227, 183)
Read: 3 size (227, 183)
Read: 4 size (227, 183)
Read: 5 size (227, 183)
Read: 6 size (227, 183)
Read: 7 size (227, 183)


### Passar a matriu i binaritzem

Consideracions a tenir en compte. Primer, treiem les 3 primeres files i columnes de cada full de càlcul perquè a l'excel contenen etiquetes, no dades. En segon lloc, tenim 227 diles i 183 columnes. El que fem és forçar una matriu quadrada nre.filesxnre.files, de manera que, com la matriu inicial eren zeros, copio les columnes existets i deixo la resta plenes de zeros. Això podria donar error. NOTA: Els datasets compten la primera columna com títols.

In [5]:
## Hipermatrix M with the data (Only the synapses) 
rows = d["group" + str(i)].shape[0] - 3
columns = d["group" + str(i)].shape[1] - 3 #les tres primeres files i columnes no contenen dades

M = np.zeros((fulls,rows,columns))
for i in range(0, fulls):
    Data = d['group' + str(i)]
    M[i,:,:] = Data.iloc[3:,3:]
    
## Since we work with same number of nodes, we want them equal and square (zeros when no connections)
M_square = np.zeros((fulls, rows, rows))#creo matriu quadrada
M_square[:,:, 0:columns] = M[:,:,:] #copies les columnes existents (0..179) per deixar zeros en les restants (180..223).

## Binarization: No weights
#Lo queremos BINARIO, ignorando su peso (Luego pensar cómo se haría con el peso)
M_square_bin = np.zeros((fulls,rows,rows))
for i in range(fulls):
    for j in range(rows):
        for k in range(columns):
            if (M[i,j,k] >= 1):
                M_square_bin[i,j,k] = 1
                
Nx = rows # Number of nodes (we imposed rows == columns)


### Comprovació de l’estructura dels fulls de càlcul

Vull assegurar-me de que la neteja dels Datasets (primeres 3 files i columnes) està feta correctament. Nota: Els Datasets prenen la primera fila com un Títol  ('Pre', en aquest cas). Per tant, realment n'estic esborrant 4.

In [6]:
# inspecció ràpida d'un full de càlcul (group 7-->Dataset8)
df7 = d['group7']
print("Shape original group7:", df7.shape)
print("\nColumn names (first 12):\n", df7.columns[:12])
print("\nPrimeres 8 files i 8 columnes (sense transformar):")
print(df7.iloc[:8, :8])

Shape original group7: (227, 183)

Column names (first 12):
 Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Pre', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

Primeres 8 files i 8 columnes (sense transformar):
  Unnamed: 0 Unnamed: 1 Unnamed: 2      Pre Unnamed: 4 Unnamed: 5 Unnamed: 6  \
0        NaN        NaN        NaN  Sensory        NaN        NaN        NaN   
1        NaN        NaN        NaN     ADFL       ADFR       ADLL       ADLR   
2        NaN        NaN        NaN      NaN        NaN        NaN        NaN   
3       Post    Sensory       ADFL        0          0          0          0   
4        NaN        NaN       ADFR        0          0          0          0   
5        NaN        NaN       ADLL        0          0          0          0   
6        NaN        NaN       ADLR        0          0          0          0   
7        NaN        NaN       AFDL      

## Escollir quin és el dataset A2

Sabem (per la informació del paper) que A2 té aproximadament 2186 arestes (edges), comprovem si el Dataset8 concorda amb aquestes dades, mirem cada dataset (recordem que group0-->Dataset1, group1-->Dataset2, ...).

NOTA: A més sospitem que el Dataset8 és el que conté les dades de A2 perquè al repo de la Teresa es pren A2 com a Blueprint fent M_square_bin[-1] que correspon a l'últim element de l'array, és a dir, M_square_bin[7], que correspon a group7--> Dataset8

### Coprovació de que Dataset8 correspon a A2

In [7]:
# Veure quants edges té cada dataset
for i in range(8):
    print(i, M_square_bin[i].sum())

0 775.0
1 986.0
2 1012.0
3 1136.0
4 1515.0
5 1525.0
6 2202.0
7 2186.0


## Guardar A2 com blueprint

Volem crear i guardar el blueprint A2 (Dataset8 = group7 = M_square_bin[-1])


In [8]:
#Seleccionem A2, és a dir, el dataset8
A2=M_square_bin[-1].copy()
#Fem comprovacions
print("A2 creat")
print("Files i columnes A2", A2.shape)
print("Arestes", int(A2.sum()))
#guardem
np.save("A2_blueprint.npy", A2)
print("Guardat com A2_blueprint")

A2 creat
Files i columnes A2 (224, 224)
Arestes 2186
Guardat com A2_blueprint


# Generar xarxes amb error

Dues maneres, fixar un nombre d'errors (com es fa al codi de la Teresa) o fer servir p i q (probabilitats de copiar malament un link/no-link). Farem totes dues

### ESBORRAR AIXÒ: Veure d'on sortien els errors q feia servir Teresa

In [9]:
L = np.load("A2_blueprint.npy")
Nx = L.shape[0]
edges = int(L.sum())
zeros = Nx*Nx - edges
e_error = 327
z_error = 327
q_equiv = e_error / edges
p_equiv = z_error / zeros
print("edges", edges, "zeros", zeros)
print("q_equiv =", q_equiv)
print("p_equiv =", p_equiv)

edges 2186 zeros 47990
q_equiv = 0.1495882891125343
p_equiv = 0.00681391956657637


Hem vist que p i q del repositori son practicament els mateixos del paper

### Càlcul de nombre d'errors equivalents per p i q

In [10]:
# Carrega el blueprint A2 desat
L = np.load("A2_blueprint.npy")

Nx = L.shape[0]

print("Blueprint A2 carregat correctament.")
print("Mida:", L.shape)
print("Nombre d'enllaços (1’s):", int(L.sum()))


Blueprint A2 carregat correctament.
Mida: (224, 224)
Nombre d'enllaços (1’s): 2186


In [11]:
# Errors deduits dels valors del paper
edges_A2 = L.sum()
zeros_A2 = Nx*Nx - edges_A2

q_paper = 0.15
p_paper = 0.007

errors_esborrar = int(q_paper * edges_A2)
errors_crear = int(p_paper * zeros_A2)

print(errors_esborrar, errors_crear)


327 335


No són exactament els mateixos, suposo que això es deu a què al repo es fan servir p i q que no quadren exactament amb els del paper. 

(Esborrar això) ara faré el mateix però amb els p i q que he deduit del codi de la teresa, per veure si em surt el mateix nre, d'errors que ella ha fet servir (327,327) i comprovar validesa de l'anterior

In [12]:
# Errors deduits dels valors del paper
edges_A2 = L.sum()
zeros_A2 = Nx*Nx - edges_A2

errors_esborrar_teresa = int(q_equiv * edges_A2)
errors_crear_teresa = int(p_equiv * zeros_A2)

print(errors_esborrar_teresa, errors_crear_teresa)


327 327


Sí, el nombre d'errors quadra

# Generar xarxes amb nombre d'errors

Ara estem generant xarxes com es feia al repositori, amb el mateix nombre d'errors exacte

In [17]:
# === Generar xarxes amb errors exactes (estil del repo) ===

K = 4               # nombre de xarxes a generar
e_error = 327       # errors esborrant enllaços
z_error = 327       # errors creant enllaços

# Carregar blueprint si cal
if 'L' not in globals():
    print("No hi ha L a la memòria, carregant A2_blueprint.npy ...")
    L = np.load("A2_blueprint.npy")

Nx = L.shape[0]
print("Blueprint A2 carregat: shape", L.shape, "edges:", int(L.sum()))

# Preparar output
A_repo = np.zeros((K, Nx, Nx), dtype=int)
rng = np.random.default_rng(42)  # llavor fixa

edges_total = int(L.sum())
zeros_total = Nx * Nx - edges_total

print("Edges A2:", edges_total, "Zeros A2:", zeros_total)

# Comprovacions
if e_error > edges_total:
    raise ValueError(f"e_error ({e_error}) > nombre d'enllaços ({edges_total})")
if z_error > zeros_total:
    raise ValueError(f"z_error ({z_error}) > nombre de zeros ({zeros_total})")

for m in range(K):
    A = L.copy().astype(int)
    A_flat = A.reshape(Nx*Nx)

    ones_idx = np.where(A_flat == 1)[0]
    zeros_idx = np.where(A_flat == 0)[0]

    flip_ones = rng.choice(ones_idx, size=e_error, replace=False)
    flip_zeros = rng.choice(zeros_idx, size=z_error, replace=False)

    A_flat[flip_ones] = 0
    A_flat[flip_zeros] = 1

    A = A_flat.reshape(Nx, Nx)
    A_repo[m] = A

    errors_removed = edges_total - np.sum((A == 1) & (L == 1))
    errors_added = np.sum((A == 1) & (L == 0))

    print(f"Xarxa {m}: Edges = {int(A.sum())}, Errors esborrant = {errors_removed}, Errors creant = {errors_added}")

# Guardar a disc
out_name = "synthetic_repo_errors.npy"
np.save(out_name, A_repo)
print("\nGuardades les xarxes amb errors exactes a:", out_name)


Blueprint A2 carregat: shape (224, 224) edges: 2186
Edges A2: 2186 Zeros A2: 47990
Xarxa 0: Edges = 2186, Errors esborrant = 327, Errors creant = 327
Xarxa 1: Edges = 2186, Errors esborrant = 327, Errors creant = 327
Xarxa 2: Edges = 2186, Errors esborrant = 327, Errors creant = 327
Xarxa 3: Edges = 2186, Errors esborrant = 327, Errors creant = 327

Guardades les xarxes amb errors exactes a: synthetic_repo_errors.npy


# Generar xarxes amb p i q

In [15]:
num_networks = 6
A_test = np.zeros((num_networks, Nx, Nx))

p = p_paper
q = q_paper

np.random.seed(1)

for k in range(num_networks):
    A_copy = L.copy()
    for i in range(Nx):
        for j in range(Nx):
            if L[i,j] == 1:
                # enllaç existent -> pot ser esborrat
                if np.random.rand() < q:
                    A_copy[i,j] = 0
            else:
                # posició buida -> pot crear-se un fals
                if np.random.rand() < p:
                    A_copy[i,j] = 1
    A_test[k,:,:] = A_copy

print("Xarxes generades amb p i q del paper:")
print(A_test.shape)
np.save("synthetic_paper.npy", A_test)


Xarxes generades amb p i q del paper:
(6, 224, 224)


Ara volem comprovar el nombre d'enllaços que s'han creat (p·zeros ≈ 0.007·48000 ≈ 330–350 nous) i esborrat (q·edges ≈ 0.15·2186 ≈ 327–330 enllaços)

In [16]:
# Comprovació del nombre d’enllaços final
original_edges = int(L.sum())
zeros_original = Nx*Nx - original_edges

print("Blueprint (A2) edges:", original_edges)
print("Blueprint zeros:", zeros_original)

for k in range(num_networks):
    edges_new = int(A_test[k].sum())
    print(f"\nXarxa {k}:")
    print("  Edges:", edges_new)

    # errors equivalents
    errors_esborrats = original_edges - np.sum((A_test[k] == 1) & (L == 1))
    errors_creats = np.sum((A_test[k] == 1) & (L == 0))

    print("  Errors esborrant:", errors_esborrats)
    print("  Errors creant:", errors_creats)


Blueprint (A2) edges: 2186
Blueprint zeros: 47990

Xarxa 0:
  Edges: 2207
  Errors esborrant: 320
  Errors creant: 341

Xarxa 1:
  Edges: 2174
  Errors esborrant: 333
  Errors creant: 321

Xarxa 2:
  Edges: 2175
  Errors esborrant: 333
  Errors creant: 322

Xarxa 3:
  Edges: 2190
  Errors esborrant: 329
  Errors creant: 333

Xarxa 4:
  Edges: 2179
  Errors esborrant: 304
  Errors creant: 297

Xarxa 5:
  Edges: 2169
  Errors esborrant: 334
  Errors creant: 317
